In [1]:
import numpy as np
import pandas as pd

In [3]:
data=pd.read_csv('stroke_data.csv')

In [5]:
data.head()

,Age,Gender,SES,Hypertension,Heart_Disease,BMI,Avg_Glucose,Diabetes,Smoking_Status,Stroke
0,66.788137,Male,Medium,1,0,25.842521,96.428681,1,Current,1
1,86.393609,Female,Medium,1,1,32.531061,133.350068,0,Never,1
2,76.158579,Female,Medium,1,0,40.681244,111.489589,0,Never,0
3,72.477870,Female,Low,0,1,33.003193,125.692465,0,Former,0
4,59.881635,Male,Low,0,0,26.062443,123.218486,1,Never,1


In [7]:
data.isnull().sum()

Age               0
Gender            0
SES               0
Hypertension      0
Heart_Disease     0
BMI               0
Avg_Glucose       0
Diabetes          0
Smoking_Status    0
Stroke            0
dtype: int64

In [9]:
df=data.copy()

In [11]:
df['Gender']=df['Gender'].map({'Male':1,'Female':0})

In [13]:
df['SES']=df['SES'].map({'Low':-1,'Medium':0,'High':1})

In [15]:
df['Smoking_Status']=df['Smoking_Status'].map({'Current':-1,'Former':1,'Never':1})

In [17]:
df.head()

,Age,Gender,SES,Hypertension,Heart_Disease,BMI,Avg_Glucose,Diabetes,Smoking_Status,Stroke
0,66.788137,1,0,1,0,25.842521,96.428681,1,-1,1
1,86.393609,0,0,1,1,32.531061,133.350068,0,1,1
2,76.158579,0,0,1,0,40.681244,111.489589,0,1,0
3,72.477870,0,-1,0,1,33.003193,125.692465,0,1,0
4,59.881635,1,-1,0,0,26.062443,123.218486,1,1,1


In [19]:
X=df.drop('Stroke',axis=1)

In [21]:
y=df['Stroke']

In [105]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import MinMaxScaler

In [27]:
cols=X.columns

In [29]:
scaler=MinMaxScaler()

In [31]:
X=scaler.fit_transform(X)

In [33]:
X=pd.DataFrame(data=X,columns=cols)

In [35]:
X.head()

,Age,Gender,SES,Hypertension,Heart_Disease,BMI,Avg_Glucose,Diabetes,Smoking_Status
0,0.544988,1.0,0.5,1.0,0.0,0.332878,0.389395,1.0,0.0
1,0.818384,0.0,0.5,1.0,1.0,0.538945,0.672076,0.0,1.0
2,0.675658,0.0,0.5,1.0,0.0,0.790045,0.504706,0.0,1.0
3,0.624331,0.0,0.0,0.0,1.0,0.553491,0.613447,0.0,1.0
4,0.448678,1.0,0.0,0.0,0.0,0.339653,0.594506,1.0,1.0


In [37]:
X.SES.value_counts()

SES
0.5    5106
0.0    2870
1.0    2024
Name: count, dtype: int64

In [41]:
from sklearn.linear_model import LogisticRegression

In [43]:
log=LogisticRegression(solver='liblinear')

In [45]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)

In [47]:
log.fit(X_train,y_train)

LogisticRegression(solver='liblinear')

In [49]:
log_pred=log.predict(X_test)

In [217]:
from sklearn.metrics import classification_report,roc_auc_score,accuracy_score

In [53]:
roc_auc_score(y_test,log_pred)

0.734497377810883

In [55]:
print(classification_report(y_test,log_pred))

              precision    recall  f1-score   support

           0       0.82      0.91      0.86      1377
           1       0.73      0.56      0.64       623

    accuracy                           0.80      2000
   macro avg       0.78      0.73      0.75      2000
weighted avg       0.79      0.80      0.79      2000



In [59]:
data.Stroke.value_counts()/len(data)

Stroke
0    0.7022
1    0.2978
Name: count, dtype: float64

In [63]:
from sklearn.ensemble import RandomForestClassifier

In [97]:
rfc=RandomForestClassifier(max_depth=4,criterion='entropy',n_estimators=200)

In [99]:
rfc.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=4, n_estimators=200)

In [100]:
frc_pred=rfc.predict(X_test)

In [101]:
roc_auc_score(y_test,frc_pred)

0.6981358502618693

In [107]:
param={'max_depth':[2,3,4,5,6],
      'max_features':['auto','sqrt','log2',None],
      'n_estimators':[100,200,300,400,500],
      'criterion':['gini','entropy']}

In [109]:
grd=GridSearchCV(rfc,cv=3,param_grid=param,n_jobs=4)

In [111]:
grd.fit(X_train,y_train)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
150 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
68 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\ProgramData\anaconda3\Lib\site-package

GridSearchCV(cv=3,
             estimator=RandomForestClassifier(criterion='entropy', max_depth=4,
                                              n_estimators=200),
             n_jobs=4,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5, 6],
                         'max_features': ['auto', 'sqrt', 'log2', None],
                         'n_estimators': [100, 200, 300, 400, 500]})

In [113]:
grd.best_params_

{'criterion': 'gini',
 'max_depth': 6,
 'max_features': 'log2',
 'n_estimators': 200}

In [115]:
rfc.set_params(criterion='gini',max_depth=6,max_features='log2',n_estimators=200)

RandomForestClassifier(max_depth=6, max_features='log2', n_estimators=200)

In [117]:
rfc.fit(X_train,y_train)

RandomForestClassifier(max_depth=6, max_features='log2', n_estimators=200)

In [119]:
rfc_gr_pred=rfc.predict(X_test)

In [121]:
roc_auc_score(y_test,rfc_gr_pred)

0.7139377598729879

In [219]:
accuracy_score(y_test,rfc_gr_pred)

0.7955

In [221]:
print(classification_report(y_test,rfc_gr_pred))

              precision    recall  f1-score   support

           0       0.80      0.93      0.86      1377
           1       0.76      0.50      0.60       623

    accuracy                           0.80      2000
   macro avg       0.78      0.71      0.73      2000
weighted avg       0.79      0.80      0.78      2000



In [123]:
from xgboost import XGBClassifier

In [165]:
xgb=XGBClassifier(learning_rate=0.11,max_depth=6)

In [167]:
xgb.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.11, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [169]:
pred=xgb.predict(X_test)

In [171]:
roc_auc_score(y_test,pred)

0.7252296673975457

In [173]:
df_2=data.copy()

In [191]:
from sklearn.preprocessing import LabelEncoder,StandardScaler

In [177]:
encoder=LabelEncoder()

In [181]:
df_2['Gender']=encoder.fit_transform(df_2['Gender'])

In [183]:
df_2.head()

,Age,Gender,SES,Hypertension,Heart_Disease,BMI,Avg_Glucose,Diabetes,Smoking_Status,Stroke
0,66.788137,1,Medium,1,0,25.842521,96.428681,1,Current,1
1,86.393609,0,Medium,1,1,32.531061,133.350068,0,Never,1
2,76.158579,0,Medium,1,0,40.681244,111.489589,0,Never,0
3,72.477870,0,Low,0,1,33.003193,125.692465,0,Former,0
4,59.881635,1,Low,0,0,26.062443,123.218486,1,Never,1


In [185]:
df_2.Gender.value_counts()

Gender
0    5037
1    4963
Name: count, dtype: int64

In [187]:
df_2['Smoking_Status']=encoder.fit_transform(df_2['Smoking_Status'])

In [203]:
df_2['SES']=encoder.fit_transform(df_2['SES'])

In [189]:
df_2.Smoking_Status.value_counts()

Smoking_Status
2    5951
0    2062
1    1987
Name: count, dtype: int64

In [193]:
Sscaler=StandardScaler()

In [207]:
X_1=df_2.drop('Stroke',axis=1)

In [209]:
y_2=df_2['Stroke']

In [211]:
X_1=Sscaler.fit_transform(X_1)

In [213]:
X_1=pd.DataFrame(data=X_1)

In [215]:
X_1.head()

,0,1,2,3,4,5,6,7,8
0,-0.300914,1.007428,0.879998,0.806666,-0.659021,-0.448817,-0.772655,2.025350,-1.722643
1,1.685663,-0.992627,0.879998,0.806666,1.517402,0.915863,1.475863,-0.493742,0.757943
2,0.648571,-0.992627,0.879998,0.806666,-0.659021,2.578766,0.144557,-0.493742,0.757943
3,0.275614,-0.992627,-0.392043,-1.239671,1.517402,1.012194,1.009514,-0.493742,-0.482350
4,-1.000733,1.007428,-0.392043,-1.239671,-0.659021,-0.403946,0.858848,2.025350,0.757943
